# Functions to determine R-values

In [1]:
from model.modelhelper import *

## Random Walk

4


In [7]:
class Transformations:
    def __init__(self):
        pass
    def bij_transform(self, prime, lower, upper):
      """Recieves a prime value of type tensor in [-inf, inf] and returns value in [lower, upper]"""
      bij = 1 / (1 + torch.exp(-prime / upper))
      scale = upper - lower
      return scale * bij + lower

    def bij_transform_inv(self, transf, lower, upper):
      """Inverse transformation - Recieves a value of type tensor in [lower, upper] and returns value in [-inf, inf]"""
      return -torch.log(((upper - lower) / (transf - lower) - 1) ** upper)


class RandomWalk(Transformations):
    def __init__(self, sigma_init, r_0, lower, upper, device, dtype):
        self.R0 = r_0
        self.sigma = sigma_init
        self.sigma_prime = self.bij_transform_inv(torch.tensor(self.sigma, device=device, dtype=dtype), lower, upper).detach().clone().requires_grad_(True)



In [ ]:
def random_walk(epsilon, R0 , device, dtype):
    n = epsilon.size(0)
    # Initialize eta
    eta = torch.zeros(n, device=device, dtype=dtype)  # transformed reproduction number
    # calculate Rt: the basic reproduction number
    # basic reproduction number as a latent random walk
    beta_0 = torch.log(R0)
    eta[0] = beta_0
    eta[1:n] =beta_0 + epsilon[0:n-1]
    R = torch.exp(eta)
    return R